In [11]:
import tensorflow as tf
import matplotlib.pyplot as pls
import os

from tensorflow import keras 

In [26]:
# Global Constants
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512
CHANNELS = 3
BATCH_SIZE = 2
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512

# File Paths
image_path = os.path.join("data", "images")
mask_path = os.path.join("data", "masks")

## The arguments for keras image data generator functions
# For the training and validation sets
TRAIN_ARGS = dict(
    rescale=1.0/255,        # Rescaling factor, applied after all other transformations
    shear_range=0.1,        # Shear the Image by 10%
    zoom_range=0.1,         # Zoom in or out by 10%
    horizontal_flip=True,   # Randomly flip horizontally
    vertical_flip=True,     # Randomly flip vertically
    fill_mode='nearest',    # Fill gaps with nears pixel
    validation_split=0.2)   # Reserve 20% for 'validation'

# For the testing split
TEST_ARGS = dict(
    rescale=1.0/255,        # Rescaling factor, applied after all other transformations
    validation_split=0.8)   # Reserve 80% for 'validation' (training)

# Arguments for the flow from directory function call
IMAGE_LOAD_ARGS = dict(
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    interpolation="nearest",
    subset='training',
    class_mode=None,
    seed=42
    ) 

In [38]:
def pre_processing():
    # Create image data generator that applies the data augmentation detailed in constants
    train_image_generator = keras.preprocessing.image.ImageDataGenerator(**TRAIN_ARGS)
    train_mask_generator = keras.preprocessing.image.ImageDataGenerator(**TRAIN_ARGS)
    test_image_generator = keras.preprocessing.image.ImageDataGenerator(**TEST_ARGS)
    test_mask_generator = keras.preprocessing.image.ImageDataGenerator(**TEST_ARGS)
    
    # Load test/train data through the respective data generator
    # Training data 
    image_train = train_image_generator.flow_from_directory(
        directory=image_path,
        color_mode="rgb",
        **IMAGE_LOAD_ARGS)
    
    # Training Mask Data
    mask_train = train_mask_generator.flow_from_directory(
        directory=mask_path,
        color_mode="grayscale",
        **IMAGE_LOAD_ARGS)
    
    
    # Test Data
    image_test = test_image_generator.flow_from_directory(
        directory=image_path,
        color_mode="rgb",
        **IMAGE_LOAD_ARGS)
    
    # Test Mask Data
    mask_test = test_mask_generator.flow_from_directory(
        directory=mask_path,
        color_mode="grayscale",
        **IMAGE_LOAD_ARGS)
    
    # Return a joined version of the training data (image,mask) and a joined version of the test data (image,mask)
    return zip(image_train, mask_train), zip(image_test, mask_test)
    

In [39]:
pre_processing()

Found 2076 images belonging to 1 classes.
Found 2076 images belonging to 1 classes.
Found 519 images belonging to 1 classes.
Found 519 images belonging to 1 classes.


(<zip at 0x1d089a06100>, <zip at 0x1d08b0f5880>)